<a href="https://colab.research.google.com/github/saeu5407/daily_tensorflow/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2seq

seq2seq를 활용해 기계번역을 진행해보도록 하겠습니다.


In [1]:
pip install tensorflow_text

     |████████████████████████████████| 4.4 MB 4.3 MB/s 


In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# 모양을 확인하는 데 사용하기 위해 정의해 둔 저수준 API 관련 옵션
use_builtins = True

하단 코드는 모양 검사를 위한 코드입니다.

```
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    if isinstance(names, str):
      names = (names,)

    shape = tf.shape(tensor)
    rank = tf.rank(tensor)

    if rank != len(names):
      raise ValueError(f'Rank mismatch:\n'
                       f'    found {rank}: {shape.numpy()}\n'
                       f'    expected {len(names)}: {names}\n')

    for i, name in enumerate(names):
      if isinstance(name, int):
        old_dim = name
      else:
        old_dim = self.shapes.get(name, None)
      new_dim = shape[i]

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")
```

## 데이터셋 로드 및 준비

텐서플로우 튜토리얼을 따라 영어-스페인어 데이터 세트를 사용해 데이터셋을 로드합니다.
해당 데이터세트는 다음과 같이 한 쌍으로 구성되어 있습니다.

```
May I borrow this book? ¿Puedo tomar prestado este libro?
```

다운받은 데이터에 대해 다음과 같은 작업을 진행합니다.
1. 각 문장에 토큰 시작과 끝을 추가
2. 특수 문자 제거
3. 단어 -> id, id -> 단어 등 사전 매핑
4. 각 문장을 최대 길이로 채움

In [ ]:
# 데이터 다운로드
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

In [ ]:
# 데이터 로드 함수 정의
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines() # 여러 라인으로 구분된 문자열을 한 라인씩 분리하여 리스트 반환
  pairs = [line.split('\t') for line in lines] # 그 리스트를 탭으로 분리하여 영어, 스페인 문장을 나눔

  inp = [inp for targ, inp in pairs]
  targ = [targ for targ, inp in pairs]

  return targ, inp

In [ ]:
targ, inp = load_data(path_to_file)
print(inp[-1])